# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [10]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [11]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Tech Linkedin Content Creator tasked with generating new AI news."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
        "The output only shows the post content."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [12]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Write a 100-word linkedin post content explaining the difference between Deep Learning and Machine Learning for a non-technical audience."
    }
)

Let's generate the first version of the essay.

In [13]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

InternalServerError: Error code: 503 - {'error': {'message': 'Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.', 'type': 'internal_server_error'}}

In [ ]:
display_markdown(mergesort_code, raw=True)

"Unravel the Mystery: Deep Learning vs Machine Learning

Ever wondered what sets these two buzzwords apart? 

Machine Learning is like a super-smart personal assistant that learns from data to make predictions or decisions. It's a type of Artificial Intelligence that enables computers to learn from experience.

Deep Learning, on the other hand, is a subset of Machine Learning that's inspired by the human brain. It uses neural networks to analyze data and make decisions, much like how our brains process information. Think self-driving cars, facial recognition, and language translation!

In short, Machine Learning is the umbrella, and Deep Learning is a powerful tool underneath. #AI #MachineLearning #DeepLearning"

## Reflection Step

We will go for reflection step to make sure the content is attractive enough for a linkedin post.<br>
We are going to use one-shot prompt technique for the agent.

In [ ]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are an expert LinkedIn marketing analyst. Your role is to critique the provided LinkedIn post on two key areas: Creativity and Format.",
    }
]

We are going to extend the prompt with one-shot-prompting to have a better critique generation

In [ ]:
reflection_chat_history.extend([
    {
        "role": "user",
        "content": 
        """
        # Example content for critique:
        AI is changing everything. If you're not using AI tools you are missing out. I think everyone should learn how to use AI in their job because it makes you so much more productive and it's the future of work. Companies are looking for people with AI skills.
        #AI #FutureOfWork #Tech #Innovation #Productivity #ArtificialIntelligence #CareerDev
        """
            },
            {
                "role": "assistant",
                "content": """
        # Example Critique:
        Here is a format critique of your LinkedIn post:
        Readability: The post is a single, large block of text, which is very difficult to read on a screen. It needs to be broken down into short, scannable sentences and paragraphs, with plenty of whitespace in between to make it more inviting.
        Visuals: The post lacks any visual elements. Adding 2-3 professional emojis (like 💡, 🚀, or 📈) would make the key points stand out and add a touch of personality without looking unprofessional.
        Hashtags: There are too many hashtags (7), and some are too generic. This can look spammy. It's better to stick to 3-5 highly relevant and specific hashtags.
        """
    }
])


The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [ ]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [ ]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

InternalServerError: Error code: 503 - {'error': {'message': 'Service unavailable. Visit https://groqstatus.com/ to see if there is an active incident.', 'type': 'internal_server_error'}}

In [ ]:
display_markdown(critique, raw=True)

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [ ]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [ ]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [ ]:
display_markdown(essay, raw=True)

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

In [14]:
import sys
print(os.getcwd()) 
module_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(module_path)
# Solve the path problem

c:\UTM\Self_Improvement\AI_agent\agentic-patterns\notebooks


Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [15]:
from agentic_patterns import ReflectionAgent

In [16]:
agent = ReflectionAgent()

In [17]:
generation_system_prompt = "You are a Tech Linkedin Content Creator tasked with generating new AI news."

reflection_system_prompt = "You are an expert LinkedIn marketing analyst."

user_msg = "Write a 100-word linkedin post content explaining the difference between Deep Learning and Machine Learning for a non-technical audience."

In [18]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10

 

GENERATION

 "Unlock the power of AI. Ever wondered what sets Deep Learning apart from Machine Learning? Simply put, Machine Learning is a type of AI that enables systems to learn from data, whereas Deep Learning is a subfield of Machine Learning that uses neural networks to mimic human brain function. Think of it like a car: Machine Learning is the engine, while Deep Learning is the turbocharger that takes it to the next level. Understand the basics to unlock new possibilities in tech #AI #MachineLearning #DeepLearning"
 

REFLECTION

 Here is a critique of your LinkedIn post:

Creativity: 
The use of the car analogy to explain the difference between Machine Learning and Deep Learning is creative and helps to make the concept more accessible to a wider audience.

Format: 
The post is well-structured and easy to read. The use of a brief introduction, a clear explanation, and a concluding sentence works well. 

However, here are some recommendations for improvement:
- Con

## Final result

In [ ]:
display_markdown(final_response, raw=True)